# Point Correspondence
This notebook includes a visualization and explanation of the correspondences generation step on the Fast-Global-Registration algorithm.

<b>NOTE</b>: The visualizations open a new window in your computer, in order to continue running, the visualization window have to be closed. 

## Setup
Run the following cells to set up the notebook

In [34]:
import os
import sys

import open3d as o3d
from ipywidgets import widgets

sys.path.append("..")
from demo.utils.visualizers import visualize_correspondences
from src.logic.correspondence import collect_all_correspondences, reciprocity_test, tuple_test
sys.path.pop()

'..'

### Choose an Example
Before running the next cells, you can choose any of the examples provided in the dataset folder using the following dropdown selector.

In [39]:
dropdown = widgets.Dropdown(
    options=os.listdir('../dataset'),
    description='Example:',
    value="pairwise_no_noise_01_rot_05",
    layout={'width': 'max-content'},
    disabled=False,
)
dropdown

Dropdown(description='Example:', index=50, layout=Layout(width='max-content'), options=('pairwise_noise_xyz_le…

In [40]:
pcd1 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0000.ply")
pcd2 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0001.ply")

In [41]:
pcd1.paint_uniform_color([1, 0, 0])
pcd2.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([pcd1, pcd2], window_name="Original Point Clouds")

In [30]:
kappa1 = collect_all_correspondences(pcd1, pcd2)
print(f"Found {len(kappa1)} matches")

found 22238 matches


In [21]:
visualize_correspondences(pcd1, pcd2, kappa1, "All Correspondences")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [31]:
kappa2 = reciprocity_test(kappa1)
print(f"There are {len(kappa2)} after the reciprocity test")

There are 2083 after the reciprocity test


In [32]:
visualize_correspondences(pcd1, pcd2, kappa2, "After Reciprocity Test")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [35]:
kappa3 = tuple_test(kappa2, pcd1, pcd2)
print(f"There are {len(kappa3)} after the tuple test")

There are 992 after the tuple test


In [36]:
visualize_correspondences(pcd1, pcd2, kappa3, "After Tuple Test")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
